# Setup Minedojo Environment

In [1]:
from Environments import SkyRunner, MultithreadGym
frame_stack = False
frames = 4
use_grayscale=False

# Multithreaded environment wrapper
env = MultithreadGym.MultithreadGym(thread_int=1, env_int=1, frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)

# Evaluation environment
eval_env = SkyRunner.CustomEnv(frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)
eval_env.reset()

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader 0
ThreadID: 0 has received an enviornment from queue. Reset of environement is being prepeared


[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.
[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  6,   6,   7, ...,  12,  11,  10],
        ...,
        [ 61,  64,  19, ...,  72,  69,  65],
        [ 58,  62,  18, ...,  70,  66,  62],
        [ 56,  59,  16, ...,  68,  64,  60]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  8,   9,  10, ...,  17,  16,  14],
        ...,
        [ 72,  76,  16, ...,  85,  81,  77],
        [ 69,  73,  15, ...,  82,  78,  73],
        [ 66,  70,  12, ...,  79,  75,  70]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  2,   2,   3, ...,   5,   4,   4],
        ...,
        [ 91,  96,  10, ..., 107, 102,  96],
        [ 87,  92,  10, ..., 104,  98,  93],
        [ 83,  88,   8, ..., 100,  94,  89]]], dtype=uint8)

# Begin training

In [ ]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env, eval_env=eval_env, eval_freq=5000, use_prioritized_replay=False, name="kristian-baseline-3")

Waiting for ready enviornment
Ready enviornment received
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!


# Shutdown Environments

In [3]:
env.close()
eval_env.close()

stopping Reloader 0


# Load existing model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

_env = env
model = DoubleDQN.load("/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/dDQN-checkpoints/checkpoint_30000_steps.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0